In [4]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
from helpers import delta_r, delta_phi, inv_mass_3p, cos_opening_angle, inv_mass, delta_eta
from samples import signal_samples
from helpers import files_from_dir, files_from_dirs
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
local_dir = "C:\\Users\\lucas\\Desktop\\PDM\\data\\HeavyNeutrino_trilepton\\HeavyNeutrino_trilepton_M-300_V-0p01_tau_NLO_tauhFilter_TuneCP5_13TeV-madgraph-pythia8"
samples = files_from_dir(local_dir),


for file in samples[0]:
        events = NanoEventsFactory.from_root(file, schemaclass=NanoAODSchema).events()
        events['SelElectron'] = events.Electron[(events.Electron.pt > 24.) & (events.Electron.mvaFall17V2Iso_WP90 > 0.5)]
        events_1electron = events[ak.num(events.SelElectron) == 1]
        
        events['SelMuon'] = events.Muon[(events.Muon.pt > 24.) & (events.Muon.mediumPromptId) & (events.Muon.pfRelIso03_all < 0.2) & (np.abs(events.Muon.dxy) < 0.005)]

       
        muon1, electron1 = ak.unzip(ak.cartesian([events.SelMuon, events.SelElectron], nested=True))
        match1 = ak.any(muon1.jetIdx == electron1.jetIdx, axis=-1, mask_identity=False)
        notinjet1 = ak.any(muon1.jetIdx == -1) 
        events['SelMuon'] = events.SelMuon[(~(match1))]   
        events_1muon = events[ak.num(events.SelMuon) == 1]
        events_1muon =  events_1muon[ak.num(events_1muon.SelElectron) == 1]

        
        
        events['SelTau'] = events.Tau[(events.Tau.pt > 20.) & (abs(events.Tau.eta) < 2.3)& (events.Tau.idDeepTau2017v2p1VSmu > 0.5) & (events.Tau.idDeepTau2017v2p1VSe > 0.5) & (events.Tau.idDeepTau2017v2p1VSjet >=8)]
        tau2, electron2 = ak.unzip(ak.cartesian([events.SelTau, events.SelElectron], nested=True))
        match2 = ak.any(tau2.jetIdx == electron2.jetIdx, axis=-1, mask_identity=False)
        tau3, electron3 = ak.unzip(ak.cartesian([events.SelTau, events.SelElectron], nested=True))
        match3 = ak.any(tau3.jetIdx == electron3.jetIdx, axis=-1, mask_identity=False)
        notinjet2 = ak.any(tau2.jetIdx == int(-1))
        notinjet3 = ak.any(tau3.jetIdx == int(-1))
        
        events['SelTau'] = events.SelTau[((~(match2) & ~(match3)) )]  
        events_1tau = events[ak.num(events.SelTau) == 1]
        events_1tau = events_1tau[ak.num(events_1tau.SelElectron) == 1]
        events_1tau = events_1tau[ak.num(events_1tau.SelMuon) == 1]

        print("for emutau: all -> selectron (1 only) -> selmuon (1 only) -> seltau (1 only)")
        print(str(len(events)) + " -> "+ str(len(events_1electron)) + " -> " + str(len(events_1muon)) + " -> " + str(len(events_1tau)))
        print("all is not emutau nb of events !!!!")


for emutau: all -> selectron (1 only) -> selmuon (1 only) -> seltau (1 only)
96498 -> 36422 -> 6447 -> 3882
all is not emutau nb of events !!!!
